In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# load data
data = pd.read_excel('/Users/chenzhiang/Desktop/CASA/CASA0007/Written_investigation/data/global_terrorism_data.xlsx')

In [ ]:
# Keep only incidents determined to be terrorist attacks with 3 standards
data_1 = data[["eventid", "iyear", "imonth", "iday", "latitude", "longitude" ,"crit1","crit2","crit3","specificity"]]

data_2 = data_1.loc[(data_1['crit1'] == 1) & (data_1['crit2'] == 1)&(data_1['crit3'] == 1)]

# delete record without date or specific location

data_3 = data_2.loc[data_2['imonth'] != 0]
data_4 = data_3.loc[data_2['iday'] != 0]

data_5 = data_4[data_4['specificity'].isin([1, 2, 3])]

data_5.rename(columns={"iyear": "year"}, inplace=True)
data_5.rename(columns={"imonth": "month"}, inplace=True)
data_5.rename(columns={"iday": "day"}, inplace=True)

data_5['date'] = pd.to_datetime(data_5[['year', 'month', 'day']])

data_6 = data_5[['date', "latitude", "longitude"]]


data_6.rename(columns={"date": "temporal_index"}, inplace=True)
#data_6.rename(columns={"latitude": "x"}, inplace=True)
#data_6.rename(columns={"longitude": "y"}, inplace=True)


def dt64_to_float(dt64):
     year = dt64.astype('M8[Y]')
     days = (dt64 - year).astype('timedelta64[D]')
     year_next = year + np.timedelta64(1, 'Y')
     days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
     dt_float = 1970 + year.astype(float) + days / (days_of_year)
     return dt_float

data_6['date_float'] = dt64_to_float(data_6['temporal_index'].to_numpy())
data_6['temporal_index'] = data_6['date_float'].values

data_7 = data_6[['temporal_index', "latitude", "longitude"]]
data_7 = data_7.dropna()


In [ ]:
# Density plot

import plotly.express as px
fig = px.density_mapbox(data_7
                        ,lat='latitude'
                        ,lon='longitude'
                        ,range_color = [0, 300]
                        ,zoom=2
                        ,radius=15
                        ,opacity=.5
                        ,mapbox_style='open-street-map')
                        

fig.update_layout(
        title = 'Global Terrorist Attack Density Map',
        geo_scope='world'
    )
fig.show()

In [ ]:
# Preparing data for the pie plot
data_11 = data[["eventid", "iyear", "imonth", "iday", "latitude", "longitude" ,"crit1","crit2","crit3","vicinity","attacktype1_txt","success","suicide","weaptype1_txt", "targtype1_txt","natlty1_txt","nkill","INT_MISC"]]
# Keep only incidents determined to be terrorist attacks with 3 standards
data_12 = data_11.loc[(data_11['crit1'] == 1) & (data_11['crit2'] == 1)&(data_11['crit3'] == 1)]

data_13 = data_12.loc[data_12['imonth'] != 0]
data_14 = data_13.loc[data_13['iday'] != 0]

data_14.rename(columns={"iyear": "year"}, inplace=True)
data_14.rename(columns={"imonth": "month"}, inplace=True)
data_14.rename(columns={"iday": "day"}, inplace=True)

data_14['date'] = pd.to_datetime(data_14[['year', 'month', 'day']])

data_15 = data_14.dropna()

In [ ]:
# pie-plot in the city
data_16 = data_15.loc[data_15['vicinity'] != -9]

plt.clf()
data_17 = data_16[['vicinity', "eventid"]]

mylabels = ["City", "Non-city"]
myexplode = [0.2, 0]
mycolors = ["#8B0000", "#8B4513"]
plt.title('Whether Terrorist Attack Occured in the City')
plt.pie(data_17.groupby(["vicinity"])["eventid"].count(), labels = mylabels, autopct='%1.0f%%', explode = myexplode,  colors = mycolors)

plt.show() 

In [ ]:
# pie-plot attacktype

data_18 = data_15[['attacktype1_txt', "eventid"]]
data_19 = data_18.groupby(["attacktype1_txt"])["eventid"].count().reset_index()
data_19 = data_19.sort_values("eventid",ascending=[False])

plt.clf()
mycolors = ['#8B0000', '#8B4513', '#DAA520', '#2F4F4F', '#4682B4', '#191970', '#483D8B', '#696969', '#BC8F8F']
plt.title('Terrorist Attacks Type')
plt.pie(data_19['eventid'], startangle=90,  colors = mycolors, textprops={'color':"w",'size': 8},
autopct=lambda pct: '{:1.1f}%'.format(pct) if pct > 5 else '')


plt.legend(data_19["attacktype1_txt"], bbox_to_anchor=(0, 1),  prop={'size': 6.5})



plt.show() 
plt.savefig('pie_1.png', dpi=1000)

In [ ]:
# pie-plot success

data_20 = data_15[['success', "eventid"]]
data_21 = data_20.groupby(["success"])["eventid"].count().reset_index()
data_21 = data_21.sort_values("eventid")

plt.clf()
mycolors = ["#8B4513","#8B0000"]
myexplode = [0.2, 0]
mylabels = ["Fail", "Success"]
plt.title('Whether Terrorist Attacks Successfully Conducted')
plt.pie(data_21['eventid'],  autopct='%1.0f%%', startangle=-45,  labels = mylabels, explode = myexplode, colors = mycolors)
plt.show() 

In [ ]:
# pie-plot suicide

data_22 = data_15[['suicide', "eventid"]]
data_23 = data_22.groupby(["suicide"])["eventid"].count().reset_index()
data_23 = data_23.sort_values("eventid")

plt.clf()
mycolors = ["#8B0000","#8B4513"]
myexplode = [0.2, 0]
mylabels = ["Suicide Attack", "Non-Suicide Attack"]
plt.title('Whether Terrorist Attacks Suicidal')
plt.pie(data_23['eventid'],  autopct='%1.0f%%', startangle=135,  labels = mylabels, explode = myexplode, colors = mycolors)
plt.show() 

In [ ]:
# pie-plot weaptype

data_24 = data_15[['weaptype1_txt', "eventid"]]
data_25 = data_24.groupby(["weaptype1_txt"])["eventid"].count().reset_index()
data_25 = data_25.sort_values("eventid",ascending=[False])

data_26 = data_25.replace("Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)", "Vehicle")

plt.clf()
mycolors = ['#8B0000', '#8B4513', '#DAA520', '#2F4F4F', '#4682B4', '#191970', '#483D8B', '#696969', '#BC8F8F', '#008B8B', '#556B2F', '#4B0082']
plt.title('Terrorist Attacks Weapon')
plt.pie(data_26['eventid'],   startangle=90,  colors = mycolors, textprops={'color':"w",'size': 8},
autopct=lambda pct: '{:1.1f}%'.format(pct) if pct > 2 else '')
plt.legend(data_26["weaptype1_txt"], bbox_to_anchor=(0, 1),  prop={'size': 6.5})

plt.show() 
plt.savefig('pie_2.png', dpi=1000)

In [ ]:
# pie-plot targtype1_txt

data_27 = data_15[['targtype1_txt', "eventid"]]
data_28 = data_27.groupby(["targtype1_txt"])["eventid"].count().reset_index()
data_28 = data_28.sort_values("eventid",ascending=[False])

plt.clf()
mycolors = ['#8B0000', '#8B4513', '#DAA520', '#2F4F4F', '#4682B4', '#191970', '#483D8B', '#696969', '#BC8F8F', '#008B8B', '#556B2F', '#4B0082']
plt.title('Terrorist Attacks Target Type')
plt.pie(data_28['eventid'],   startangle=90,  colors = mycolors, textprops={'color':"w",'size': 8},
autopct=lambda pct: '{:1.1f}%'.format(pct) if pct > 2.5 else '')
plt.legend(data_28["targtype1_txt"], bbox_to_anchor=(0, 1),  prop={'size': 6.5})

plt.show() 
plt.savefig('pie_3.png', dpi=1000)

In [ ]:
# pie-plot natlty1_txt

data_29 = data_15[['natlty1_txt', "eventid"]]
data_30 = data_29.groupby(["natlty1_txt"])["eventid"].count().reset_index()
data_30 = data_30.sort_values("eventid",ascending=[False])

data_31 = data_30.iloc[0:22]
data_32 = data_30.iloc[22:]

new_row = {'natlty1_txt':'Others', 'eventid':sum(data_32['eventid'])}
data_33 = data_31.append(new_row, ignore_index=True)

plt.clf()
mycolors = ['#8B0000', '#8B4513', '#DAA520', '#2F4F4F', '#4682B4', '#191970', '#483D8B', '#696969', '#BC8F8F', '#008B8B', '#556B2F', '#4B0082']
plt.title('Terrorist Attacks Target Nationality')
plt.pie(data_33['eventid'], startangle=90,  colors = mycolors, textprops={'color':"w",'size': 8},
autopct=lambda pct: '{:1.1f}%'.format(pct) if pct > 3 else '')
plt.legend(data_33["natlty1_txt"], bbox_to_anchor=(0, 1),  prop={'size': 6.5})

plt.show()
plt.savefig('pie_4.png', dpi=1000)

In [ ]:
# pie-plot INT_MISC

data_34 = data_15[['INT_MISC', "eventid"]]
data_35 = data_34.groupby(["INT_MISC"])["eventid"].count().reset_index()
data_35 = data_35.sort_values("eventid")

plt.clf()
mycolors = ["#8B4513","#8B0000"]
myexplode = [0.2, 0]
mylabels = ["International", "Domestic"]
plt.title('Whether Terrorist Attacks Target International Targets')
plt.pie(data_35['eventid'],  autopct='%1.0f%%', startangle=-45,  labels = mylabels, explode = myexplode, colors = mycolors)
plt.show()

In [ ]:
# hist year
import matplotlib.pyplot as plt
data_36 = data_15[['year', "eventid"]]
data_37 = data_36.groupby(["year"])["eventid"].count().reset_index()

plt.clf()

plt.scatter(data_37['year'], data_37['eventid'],s=data_37['eventid']/2,alpha=0.3)
plt.plot(data_37['year'], data_37['eventid'],color="#F0F8FF")
plt.plot(data_37['year'], data_37['eventid'], marker="o", s=20, markeredgecolor="white", markerfacecolor="white")

plt.title('Terrorists Atttacks Along Time Since 1970')
plt.xlabel('Year')
plt.ylabel('Number of Terrorists Atttacks')
plt.grid(alpha=0.3)

plt.show() 

plt.savefig('hist_1.png', dpi=1000)

In [ ]:
# hist nkill
data_38 = data_15[['year', "nkill"]]
data_39 = data_38.groupby(["year"])["nkill"].sum().reset_index()

plt.clf()

plt.bar(data_39['year'], data_39['nkill'], color ='maroon', width = 0.4)

plt.title('Terrorists Atttacks Fatalities Along Time Since 1970')
plt.xlabel('Year')
plt.ylabel('Number of Terrorists Atttacks Fatalities')
plt.gca().yaxis.grid(alpha=0.3)


z = np.polyfit(data_39['year'], data_39['nkill'], 3)
p = np.poly1d(z)

#add trendline to plot
plt.plot(data_39['year'], p(data_39['year']),linestyle='--',linewidth=1)



plt.show() 

plt.savefig('abc.png', dpi=1000)
plt.savefig('hist_2.png', dpi=1000)

In [ ]:
# K-means
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.graph_objects as go

data_9 = data_7[['temporal_index', 'latitude', 'longitude']]
k_means_optimum = KMeans(n_clusters = 7, init = 'k-means++')
data_7['cluster_KMEANS'] = k_means_optimum.fit_predict(data_9)  
data_7['txt_cluster_KMEANS'] = data_7["cluster_KMEANS"].values.astype('str')


# plot
plt.clf()
fig = go.Figure(data=go.Scattergeo(
        lon = data_7['longitude'],
        lat = data_7['latitude'],
        mode = 'markers',
        marker_color = data_7['cluster_KMEANS'],
        marker_size= 1.2,
        showlegend = True
        ))
fig.update_layout(
        title = 'Global Terrorist Attacks K-means Clustering Map',
        geo_scope='world')
fig.show()

In [ ]:
# DBSCAN Spatio-temporal Clustering

from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
import plotly.express as px

# add scale to time dimension
data_7['temporal_index_scaled'] = data_7['temporal_index']*10/12
data_8 = data_7[['temporal_index_scaled', 'latitude', 'longitude']]

# DBSCAN
model = DBSCAN(eps=6, min_samples=3000)
data_7['cluster_DBSCAN'] = model.fit_predict(data_8)
data_7['txt_cluster_DBSCAN'] = data_7["cluster_DBSCAN"].values.astype('str')
data_8 = data_7.loc[(data_7['txt_cluster_DBSCAN'] != '-1')]

# 3d DBSCAN clustering plot
df = px.data.iris()
fig = px.scatter_3d(data_8, 
                    x='longitude', 
                    y='temporal_index', 
                    z='latitude',
                    color='txt_cluster_DBSCAN',
                    #symbol='cluster'
                    )
fig.update_traces(marker_size = 0.4)
fig.update_layout(scene_aspectmode='manual',
                  scene_aspectratio=dict(x=6, y=2, z=2))
fig.show()

In [ ]:
# data for mix effect regression
data_51 = data[["eventid", "iyear","country","country_txt","region_txt","crit1","crit2","crit3","vicinity","attacktype1_txt","success","suicide","weaptype1_txt", "targtype1_txt","natlty1_txt","nkill","INT_MISC"]]
data_52 = data_51.loc[(data_51['crit1'] == 1) & (data_51['crit2'] == 1)&(data_51['crit3'] == 1)]
data_53 = data_52.loc[data_52['iyear'] != 0]
data_53.rename(columns={"iyear": "year"}, inplace=True)
data_54 = data_53.dropna()
data_55 = data_54.groupby(["country","year"])["eventid"].count().reset_index()
data_56 = data_54.groupby(["country","year"])["nkill"].sum().reset_index()
#data_54.to_excel('attack_table.xlsx')
#data_55.to_excel('attack_number.xlsx')
#data_56.to_excel('attack_fatal.xlsx')

In [ ]:
# mix effect regression
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf
import math 
from sklearn import linear_model
import seaborn as sns

df = pd.read_csv("country_annual_data.csv")
#df["major_conflict"]=df["major_conflict"].values.astype('str')
df2 = df.dropna()
df3 = df2[['gdp','year','unemplyment','major_conflict']]
#df3.corr()

# plot
plt.clf()
sns.heatmap(df3.corr(), annot=True)
ax.figure.tight_layout()
plt.show() 
#plt.savefig('dd.png', dpi=1000)
x = df2[['gdp','year','unemplyment','major_conflict']]
y = df2['nkill_million']
x = sm.add_constant(x) # adding a constant
model_1 = sm.OLS(y, x).fit()
print(model_1.summary())

# regression summary
model_2 = smf.mixedlm("nkill_million ~ gdp+ year + unemplyment  ", groups = "major_conflict", data=df2).fit()
model_2.summary()

In [ ]:
# logistic regression data
data_80 = data[["eventid", "iyear", "imonth", "iday", "latitude", "longitude" ,"crit1","crit2","crit3","vicinity","attacktype1_txt","success","suicide","weaptype1_txt", "targtype1_txt","natlty1_txt","nkill","INT_MISC", "region_txt", "individual"]]
data_81 = data_80.loc[(data_80['crit1'] == 1) & (data_80['crit2'] == 1)&(data_80['crit3'] == 1)]

# delete record without date or specific location
data_82 = data_81.loc[data_81['imonth'] != 0]
data_83 = data_82.loc[data_82['iday'] != 0]
data_83.rename(columns={"iyear": "year"}, inplace=True)
data_83.rename(columns={"imonth": "month"}, inplace=True)
data_83.rename(columns={"iday": "day"}, inplace=True)
data_83['date'] = pd.to_datetime(data_83[['year', 'month', 'day']])

def dt64_to_float(dt64):
     year = dt64.astype('M8[Y]')
     days = (dt64 - year).astype('timedelta64[D]')
     year_next = year + np.timedelta64(1, 'Y')
     days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
     dt_float = 1970 + year.astype(float) + days / (days_of_year)
     return dt_float

data_83['date_float'] = dt64_to_float(data_83['date'].to_numpy())
data_83['date_final'] = data_83['date_float'].values
data_84 = data_83.dropna()
data_85 = data_84.loc[data_84['vicinity'] == 0]
data_85['fatal'] = data_85['nkill']>0
#data_80.to_excel('data_80.xlsx')
data_86 = data_85[[ 'attacktype1_txt', 'success', 'suicide', 'weaptype1_txt', 'targtype1_txt', 'nkill', 'INT_MISC', 'fatal', 'date_final', "region_txt", "individual"]]
data_87 = pd.get_dummies(data_86)

In [ ]:
# logistic regression model
from sklearn.model_selection import train_test_split
X = data_87[['success', 'suicide',  'INT_MISC', 'date_final', 'individual', 'attacktype1_txt_Armed Assault', 'attacktype1_txt_Assassination', 'attacktype1_txt_Bombing/Explosion', 'attacktype1_txt_Facility/Infrastructure Attack', 'attacktype1_txt_Hijacking', 'attacktype1_txt_Hostage Taking (Barricade Incident)', 'attacktype1_txt_Hostage Taking (Kidnapping)',  'attacktype1_txt_Unknown', 'weaptype1_txt_Biological', 'weaptype1_txt_Chemical', 'weaptype1_txt_Explosives', 'weaptype1_txt_Fake Weapons', 'weaptype1_txt_Firearms', 'weaptype1_txt_Incendiary', 'weaptype1_txt_Melee', 'weaptype1_txt_Other', 'weaptype1_txt_Radiological', 'weaptype1_txt_Sabotage Equipment', 'weaptype1_txt_Unknown', 'targtype1_txt_Abortion Related', 'targtype1_txt_Airports & Aircraft', 'targtype1_txt_Business', 'targtype1_txt_Educational Institution', 'targtype1_txt_Food or Water Supply', 'targtype1_txt_Government (Diplomatic)', 'targtype1_txt_Journalists & Media', 'targtype1_txt_Maritime', 'targtype1_txt_Military', 'targtype1_txt_NGO', 'targtype1_txt_Other', 'targtype1_txt_Police', 'targtype1_txt_Private Citizens & Property', 'targtype1_txt_Religious Figures/Institutions', 'targtype1_txt_Telecommunication', 'targtype1_txt_Terrorists/Non-State Militia', 'targtype1_txt_Tourists', 'targtype1_txt_Transportation', 'targtype1_txt_Unknown', 'targtype1_txt_Utilities', 'targtype1_txt_Violent Political Party', 'region_txt_Australasia & Oceania', 'region_txt_Central America & Caribbean', 'region_txt_Central Asia',  'region_txt_Eastern Europe', 'region_txt_Middle East & North Africa', 'region_txt_North America', 'region_txt_South America', 'region_txt_South Asia', 'region_txt_Southeast Asia', 'region_txt_Sub-Saharan Africa', 'region_txt_Western Europe']]
y = data_87['fatal']

# split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=16, max_iter=10000000)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(logreg.score(X_train, y_train))

# logistic regression results
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
# show logistic regression estimate coefficient
import statsmodels.api as sm

logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit(maxiter=100)
print(result.summary2())
   
ddd = pd.DataFrame([result.params, result.pvalues]).T
ddd.rename(columns={0: "coef"}, inplace=True)
ddd.rename(columns={1: "p-value"}, inplace=True)
ddd['coef_exp'] = ddd["coef"].apply(lambda x: math.exp(x))
ddd = ddd[['coef_exp', 'coef', 'p-value']]
ddd = ddd.loc[(ddd['p-value'] <= 0.05)]
dddd = ddd.loc[(ddd.coef_exp >2) | (ddd.coef_exp < 0.5)]
pd.options.display.float_format = '{:.3f}'.format
print(dddd)

In [ ]:
# logistic regression ROC curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.clf()
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=500)
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.02, 1])
plt.ylim([0.0, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('City Terrorist Attacks Logistic Model ROC')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()